## 추진배경

* 본격적으로 모델 학습을 하기 전, 데이터 이용 편의를 증진하고자 함
* 팀원들이 R이나 통계분석에는 익숙하나 파이썬 코딩에는 익숙하지않아, 최대한 모델링에 집중하도록 지원
  * 하나의 repository에서 원하는 데이터를 한번에 확인 가능
  * repository에서 바로 복사해서 사용가능한 샘플을 제공하여 쉽게 데이터로딩 가능
* 데이터를 하나의 페이지에서 통합관리(공공데이터 홈페이지 접속 등 불필요)
  * 업데이트 일자를 표기하여 얼마나 최신 데이터인지 확인 가능
  * 추가하고자 하는 데이터가 공공데이터포털의 데이터라면 쉽게 작업내역에 추가가능
    (리스트 추가권한을 주거나, discord봇 접수 등 생각해보았으나 구현시간대비 효용이 작아 직접 추가진행)
* 개인서버(NAS)에서 매일 특정시간 구동하여, github서버로 자동 push(업로드)하여 별도의 수작업없이 db 최신화

## 효과
* 특정 공공데이터 포털의 데이터를 사용하기 위해 했던 아래의 작업이 제거됨
  * 공공데이터포털에 접속하여 csv파일 저장
  * 저장해둔 csv파일 찾기
  * 해당 파일을 분석했던 jupyter파일로 찾아가 API호출주소나 키를 찾기
  * csv파일을 저장한 뒤 추가 업데이트가 되었는지 확인하고 재다운로드
  * 한국어 데이터 로딩을 위한 encoding규격 등의 숙지가 필요없음
    (저장시 규격을 지정하고, 로딩을 위한 샘플코드에 규격을 지정해둠)
* 위의 사항을 달성하기 위한 부가작업이 제거됨
  * csv파일 저장 및 업데이트 시기 기록, 서버에 업로드 및 파일주소 저장

## github repository
[관련 github레포](https://github.com/KR9268/db_datagokr)

* API개인키 등 정보가 있어 레포에는 csv파일과 현황만 저장함<br>
  (구현을 위해 필요한 db_list나 개인키 등의 json파일은 모두 ignore리스트로 관리)
* git clone해둔 개인서버 디렉토리에서 자동 실행

## [세부내용] 구현내용 & 사용한 언어/패키지 등
(1) json_load(`json`) : 개인키와 db_list 로딩을 위한 함수. 어려운 기능은 아니지만 여러번 쓰여 함수화 <br>
(2) download_from_data_go_kr_with_json(`requests`, `json`) : 공공데이터포털 API호출용 (실제 자료 + 전체/다운 현황 등 수신)<br>
(3) chk_json_status_of_data_go_kr : 전체/실제 건수 등 별도 처리용(대조하여 누락없이 전체자료 저장하기 위함)<br>
(4) update_readme : README파일에 파일 현황(데이터명/주소/링크 등) 업데이트용<br>
(5) git_push(`subprocess`) : 새롭게 추가된 모든사항(ignore대상 제외)을 자동으로 업로드(auto commit)하기 위한 용도<br>
(6) 위의 함수들을 기반으로 아래의 내용을 구현<br>
  * json으로 민감정보 등을 코드에 표기하지 않고 별도관리 (+ gitignore로 이중보안)<br>
  * 1건만 먼저 호출하여 전체건수를 확인 후, 한번에 전체 건을 호출 (10000건 지정 등 과도한 세팅값의 하드코딩 지양)<br>
  * `pandas`로 encoding cp949 지정하여 csv로 저장<br>
  * github raw링크형식을 활용하여 바로 파일로딩할 수 있도록 주소 생성(db에서 아래 형식으로 지정해둔 파일명 사용)<br>
```json
[
  {
  "name": "공공데이터포털의 데이터명",
  "base_url": "API Base Url",
  "address_get": "해당 데이터의 API주소(API목록 란에서 확인)",
  "file_name_to": "CSV로 저장할 파일명(.csv제외)"
  },
  {
  "name": "공공데이터포털의 데이터명2",
  "base_url": "API Base Url",
  "address_get": "해당 데이터의 API주소(API목록 란에서 확인)",
  "file_name_to": "CSV로 저장할 파일명(.csv제외)"
  }
]
```
  * time.sleep()으로 너무 자주 호출하여 서버에 무리가지 않도록 세팅

## 샘플코드(패키지 및 함수)

In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime
import subprocess
import os
import time

def json_load(json_path, encoding='utf-8'):
    with open(json_path, 'r', encoding=encoding) as file:
        json_data = json.load(file)
    return json_data

def request_and_to_json(url):
    response = requests.get(url)
    json_ob = json.loads(response.text)
    return json_ob

def chk_json_status_of_data_go_kr(json_obj):
    other_data = ['currentCount', 'matchCount', 'page', 'perPage', 'totalCount']
    result_dict = dict()
    
    for each_column in other_data:
        result_dict[each_column] = json_obj[each_column]  
    return result_dict 

def download_from_data_go_kr_with_json(url):
    json_ob = request_and_to_json(url)

    json_status = chk_json_status_of_data_go_kr(json_ob)
    if json_status['currentCount'] < json_status['totalCount']:
        url = url.replace('perPage=1',f'perPage={json_status["totalCount"]}')
        json_ob = request_and_to_json(url)

    return json_ob

def update_readme(new_content_list):
    # Open the README.md file in read mode
    with open('README.md', 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Find the index of the line that starts with '* 데이터 현황'
    index = next((i for i, line in enumerate(lines) if line.startswith('* 데이터 현황')), None)

    # If the line is found, remove the following lines and insert new content
    if index is not None:
        lines = lines[:index+1] # Remove the following lines
        #lines.extend(new_content) # Insert new content
        lines.extend(new_content_list) # Insert new content

    # Open the README.md file in write mode and write the updated content
    with open('README.md', 'w', encoding='utf-8') as file:
        file.writelines(lines)

def git_push():
    # Get a list of all .csv files in the current directory
    csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]

    # Stage all .csv files
    for file in csv_files:
        subprocess.call(['git', 'add', file])

    subprocess.call(['git', 'add', 'README.md'])
    # Commit the changes with a message
    subprocess.call(['git', 'commit', '-m', 'Automatic commit'])

    # Push the changes to the remote repository
    subprocess.call(['git', 'push'])

## 샘플코드(패키지 및 함수)

In [ ]:
# json load
serviceKey = json_load('option.json')['serviceKey']
db_list = json_load('db_list.json', 'cp949')

In [ ]:
# main

# 작업하기
txt_for_readme = ['\n']
for each in db_list:
    # 다운로드
    url = f"{each['base_url']}{each['address_get']}?page=1&perPage=1&serviceKey={serviceKey}"
    json_data = download_from_data_go_kr_with_json(url)
    result_data = chk_json_status_of_data_go_kr(json_data)
    
    # 저장
    if result_data['currentCount'] == result_data['totalCount']:
        pd.json_normalize(json_data['data']).to_csv(f"{each['file_name_to']}.csv",encoding='cp949', index=False)

    # 파일주소 및 이름, 업데이트시간 저장
    owner = 'KR9268'
    repo = 'db_datagokr'
    branch = 'main'
    file_path = f"{each['file_name_to']}.csv"

    url = f"https://raw.githubusercontent.com/{owner}/{repo}/{branch}/{file_path}"

    txt_for_readme.append(f"  *  {datetime.strftime(datetime.now(),'%Y-%m-%d')}업데이트 : {each['name']}\n{url}\n")
    time.sleep(1)

# 업데이트 내역과 파일 git push
update_readme(txt_for_readme)
git_push()